# Tests for Conways game of life 
### I will do basic test for the game in this file


In [ ]:
import pygame


## Making a grid

In [7]:
pygame.init()
HEIGHT, WIDTH = 500,500
# for now lets experiment with this
BLACK = (0, 0, 0)
WHITE = (200, 200, 200)
GREEN = (127,255,0)

In [ ]:
SCREEN = pygame.display.set_mode((WIDTH,HEIGHT))
clock = pygame.time.Clock()
running = True

In [ ]:
def drawGrid(screen,blockSize:int,state:dict)->dict:
    """ return state of the board"""
    state =dict()
    rect_dict=dict()
    i = 0
    for x in range(0,WIDTH,blockSize):
        for y in range(0,HEIGHT,blockSize):
            rect = pygame.Rect(x,y,blockSize,blockSize)
            rect_dict[i] = rect
            i+=1
            pygame.draw.rect(screen,color,rect,1)

    return state

In [4]:
PURPLE = (43, 25, 61)
PINK = (197, 151, 157)
BLUE = (44, 54, 94)
CYAN = (75, 143, 140)

In [8]:
from typing import List


class Grid:
    """
    Args:
    screen_ref: reference to the screen to draw
    block_size: size of the block
    screen_width: screen parameter
    screen_height: screen parameter
    underpopulation_threshhold:default = 2 Alive cell dies if neighbours less than this threshold(exclusive)
    overpopulation_threshold:default = 3 Alive cell dies if neighbours more than this threshold(exclusive)
    reproduction_threshold:default = 3 Dead cell becomes alive if surrounded by exactly this number of cells
    """

    def __init__(
        self,
        screen_ref,
        block_size,
        screen_width,
        screen_height,
        underpopulation_threshhold: int = 2,
        overpopulation_threshhold: int = 3,
        reproduction_threshhold: int = 3,
    ):
        self.screen = screen_ref
        self.block_size = block_size
        self.SCREEN_WIDTH = screen_width
        self.SCREEN_HEIGHT = screen_height
        self.rows: int = screen_height // block_size
        self.cols: int = screen_width // block_size
        self.num_rects: int = self.rows * self.cols
        self.rect_dict = dict()
        self.alive_status: List[bool] = [
            [False for _ in range(self.cols)] for _ in range(self.rows)
        ]
        self.ALIVE_COLOR = (127, 255, 0)
        self.DEAD_COLOR = (225, 225, 225)
        i: int = 0
        for y in range(0, self.SCREEN_HEIGHT, self.block_size):
            for x in range(0, self.SCREEN_WIDTH, self.block_size):
                rect = pygame.Rect(x, y, self.block_size, self.block_size)
                self.rect_dict[i] = rect
                i += 1

        self.underpopulation_threshhold = underpopulation_threshhold
        self.overpopulation_threshhold = overpopulation_threshhold
        self.reproduction_threshhold = reproduction_threshhold

        print("{} Cells created".format(self.num_rects))

    def drawGrid(self):
        for y in range(self.cols):
            for x in range(self.rows):
                rect = self.rect_dict[y + x * self.cols]
                isAlive = self.alive_status[x][y]
                color = self.ALIVE_COLOR if isAlive else self.DEAD_COLOR
                # 1 for outline , 0 for fill -> opp of this var
                pygame.draw.rect(self.screen, PURPLE, rect, int(not isAlive))

    def changeCellState(self, cell_x, cell_y):
        """toggle state of the cell"""
        self.alive_status[cell_x][cell_y] = not self.alive_status[cell_x][cell_y]

    def cell_pressed(self, click_x, click_y):
        cell_x, cell_y = click_x // self.block_size, click_y // self.block_size
        self.changeCellState(cell_x, cell_y)

    def simulate(self):
        # t,tr,r,br,b,bl,l,tl
        temp :List[bool] = [[False for _ in range(self.cols)] for _ in range(self.rows)]
        deltas = [(-1, 0),(-1, 1),(0, 1),(1, 1),(1, 0),(1, -1),(0, -1),(-1, -1),]
        for x in range(self.rows):
            for y in range(self.cols):
                neighbours:int = 0
                for dx,dy in deltas:
                    nx,ny = x+dx,y+dy
                    if 0<=nx<self.rows and 0<=ny<self.cols:
                        if self.alive_status[nx][ny]:
                            neighbours+=1
                if self.alive_status[x][y] :
                    if self.underpopulation_threshhold <= neighbours <= self.overpopulation_threshhold:
                        temp[x][y] = True
                else:
                    if neighbours == self.reproduction_threshhold:
                        temp[x][y] = True

        self.alive_status = temp

    def reset(self):
         self.alive_status = [[False for _ in range(self.cols)] for _ in range(self.rows)]




In [9]:
SCREEN = pygame.display.set_mode((WIDTH,HEIGHT))
clock = pygame.time.Clock()
running = True
board = Grid(SCREEN,25,WIDTH,HEIGHT)
simulate:bool = False
while running:
    SCREEN.fill(BLACK)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.MOUSEBUTTONDOWN:
            x,y = pygame.mouse.get_pos()
            # print(pygame.mouse.get_pos())
            board.cell_pressed(y,x)
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_s:
                simulate = True
            if event.key == pygame.K_p:
                simulate = False
            if event.key == pygame.K_r:
                board.reset()
            if event.key == pygame.K_n:
                board.simulate()

    if simulate: board.simulate()
    board.drawGrid()
    

    pygame.display.flip()

    clock.tick(3)

pygame.quit()

400 Cells created
